In [17]:
!pip install -U langchain langchain_core langchain_community langchain-text-splitters
!pip install pypdf
!pip install langchain_google_genai
!pip install bs4


In [25]:

!pip install -U langchain_community
!pip install rank-bm25


In [10]:
!pip install pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.9/745.9 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.9/280.9 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


In [4]:
# gemini setup
import getpass
import os
os.environ["GOOGLE_API_KEY"]=getpass.getpass("Enter the api key ")

Enter the api key ··········


In [12]:
import getpass
import os

pinecone_api_key = getpass.getpass("Enter your Pinecone API key: ")
pc = Pinecone(api_key=pinecone_api_key)

Enter your Pinecone API key: ··········


In [26]:
import bs4
from rank_bm25 import BM25Okapi
from langchain_core.prompts import ChatPromptTemplate , PromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.output_parsers import StrOutputParser
from pinecone import Pinecone, ServerlessSpec
from pydantic import BaseModel, Field
from typing import Optional



In [9]:
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash",temperature=0)


In [13]:
# embeddings = GoogleGenerativeAIEmbeddings(
#     model="models/text-embedding-004",
#     task_type="retrieval_document",
#     output_dimensionality=512
# )

#huggingface embeding for the pinecone
# index_name = "bge-base-index"

# if not pc.has_index(index_name):
#     pc.create_index(
#         name=index_name,
#         dimension=768,   # bge-base-en-v1.5 outputs 768-dim vectors
#         metric="cosine",
#         spec=ServerlessSpec(cloud="aws", region="us-east-1")
#     )


In [14]:
#google embeding modle for the pinecone
index_name = "gemini-index" # Rename to reflect the new model

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=768,   # Updated for text-embedding-004
        metric="cosine", # Gemini models work best with cosine or dot_product
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

##pydantic initalization and input validation


In [20]:
#pydantic initalization
class CustomerQuery(BaseModel):
    query: str = Field(
        ...,
        min_length=5,
        description="Customer question"
    )
    user_id: Optional[str] = Field(
        None,
        description="Unique user identifier"
    )
    order_id: Optional[str] = Field(
        None,
        description="Order ID if query is order related"
    )


##Validate the user input Fail fast strategy

In [21]:
def validate_customer_query(data: dict) -> CustomerQuery:
    return CustomerQuery(**data)


In [15]:
# Initialize Gemini Embedder
embedder = GoogleGenerativeAIEmbeddings(
    model="models/text-embedding-004",
    task_type="retrieval_document" # Best for indexing/upserting
)

In [ ]:

# vector = embedder.embed_query("Pinecone is a vector database")
# index = pc.Index(index_name)
# index.upsert(vectors=[("1", vector, {"text": "Pinecone is a vector database"})])


In [33]:
loader = WebBaseLoader(
    web_path="https://www.myntra.com/faqs",
    header_template={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
)

try:
    docs = loader.load()
except:
    docs = []

# Fallback to sample data if blocked
if not docs or "Site Maintenance" in docs[0].page_content:
    print("Web scraping blocked by Myntra. Using sample FAQ data for demonstration.")
    from langchain_core.documents import Document
    sample_faqs = [
        "How do I return a product? You can return most items within 30 days of delivery via the 'My Orders' section on our app or website.",
        "What is the status of my refund? Once the product reaches our warehouse and passes quality check, refunds are processed within 5-7 business days.",
        "Can I change my delivery address? You can only modify the delivery address before the order is shipped. Check 'Order Details' for the edit option.",
        "How do I track my order? A tracking link is sent via SMS once the order is dispatched. You can also track it in the 'Orders' section."
    ]
    docs = [Document(page_content=text, metadata={"source": "manual_entry"}) for text in sample_faqs]
else:
    print("Content loaded successfully from URL.")

Web scraping blocked by Myntra. Using sample FAQ data for demonstration.


In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=150
)

chunks = text_splitter.split_documents(docs)


In [29]:
# Extract raw text from web chunks
corpus = [doc.page_content for doc in chunks]

# Tokenize for BM25
tokenized_corpus = [text.lower().split() for text in corpus]


In [32]:
from rank_bm25 import BM25Okapi

if tokenized_corpus:
    bm25 = BM25Okapi(tokenized_corpus)
    print("BM25 index initialized successfully.")
else:
    bm25 = None
    print("Error: tokenized_corpus is empty. BM25 could not be initialized.")

Error: tokenized_corpus is empty. BM25 could not be initialized.


##Output Validation Pydantic


In [22]:

class SupportResponse(BaseModel):
    answer: str = Field(
        ...,
        description="Final AI-generated answer"
    )
    confidence: float = Field(
        ...,
        ge=0.0,
        le=1.0,
        description="Confidence score of the answer"
    )
    needs_human: bool = Field(
        ...,
        description="Should escalate to human support?"
    )
    source: str = Field(
        ...,
        description="dense | sparse | hybrid | web"
    )


##Confidence-Based Escalation Logic

In [23]:
def should_escalate(confidence: float, query: str) -> bool:
    critical_keywords = [
        "complaint", "legal", "refund not received",
        "fraud", "delay", "charged twice"
    ]

    if confidence < 0.65:
        return True

    if any(word in query.lower() for word in critical_keywords):
        return True

    return False


In [24]:
def build_response(answer: str, confidence: float, source: str, query: str) -> SupportResponse:
    return SupportResponse(
        answer=answer,
        confidence=confidence,
        needs_human=should_escalate(confidence, query),
        source=source
    )


In [34]:
# Upsert Dense Vectors to Pinecone
index = pc.Index(index_name)

if chunks:
    vectors = []
    for i, chunk in enumerate(chunks):
        # Use the embedder to generate vector
        v = embedder.embed_query(chunk.page_content)
        # Metadata stores the original text for retrieval
        vectors.append((str(i), v, {"text": chunk.page_content}))

    index.upsert(vectors=vectors)
    print(f"Successfully upserted {len(vectors)} chunks to Pinecone index: {index_name}")
else:
    print("No chunks available to upsert. Please run the document processing cells.")

No chunks available to upsert. Please run the document processing cells.


In [35]:
def hybrid_search(query: str, k: int = 2):
    # 1. Sparse Search using BM25
    query_tokens = query.lower().split()
    sparse_results = []
    if bm25:
        bm25_scores = bm25.get_scores(query_tokens)
        import numpy as np
        top_n_idx = np.argsort(bm25_scores)[-k:][::-1]
        sparse_results = [corpus[i] for i in top_n_idx if bm25_scores[i] > 0]

    # 2. Dense Search using Pinecone
    query_vector = embedder.embed_query(query)
    dense_response = index.query(vector=query_vector, top_k=k, include_metadata=True)
    dense_results = [match['metadata']['text'] for match in dense_response['matches']]

    return sparse_results, dense_results

In [36]:
def generate_support_answer(user_input: dict):
    # 1. Validate Input
    customer_query = validate_customer_query(user_input)

    # 2. Retrieve context
    sparse, dense = hybrid_search(customer_query.query)
    unique_context = list(set(sparse + dense))
    context_text = "\n".join(unique_context)

    # 3. Prompt Template
    prompt = ChatPromptTemplate.from_template("""
    You are a Myntra Customer Support Assistant.
    Use the following retrieved context to answer the customer's question.
    If the answer isn't in the context, politely state you don't have that information and suggest contacting human support.

    Context:
    {context}

    Question: {question}
    """)

    # 4. Chain execution
    chain = prompt | llm | StrOutputParser()
    answer = chain.invoke({"context": context_text, "question": customer_query.query})

    # 5. Simple confidence logic (based on retrieval match)
    confidence = 0.9 if unique_context else 0.3

    return build_response(
        answer=answer,
        confidence=confidence,
        source="hybrid" if unique_context else "none",
        query=customer_query.query
    )

# Test the full RAG pipeline
test_query = {"query": "How do I get my refund back?"}
final_response = generate_support_answer(test_query)
print(final_response.model_dump_json(indent=2))

{
  "answer": "I apologize, but the provided context does not contain information on how to get your refund back.\n\nFor assistance with your refund, please contact our human support team, and they will be happy to help you.",
  "confidence": 0.3,
  "needs_human": true,
  "source": "none"
}
